In [1]:
import pandas as pd 

In [106]:
df_sales_details = pd.read_csv('2022-04-01T12_df_sales_detail.csv', parse_dates = ['date'])
df_sales = pd.read_csv('2022-04-01T12_df_sales.csv', parse_dates = ['date'])
df_web_logs = pd.read_csv('2022-04-01T12_df_web_logs.csv', parse_dates = ['date'])

## Task 1

Определите топ-3 товара по суммарной выручке. Для решения используйте данные из файла df_sales_details

In [14]:
(
    df_sales_details
    .groupby('good', as_index=False)
    .aggregate(total_price = ('price', 'sum'))
    .sort_values('total_price', ascending=False)
    .head(3)
)

,good,total_price
3,chefs pizza,24558240
6,double pepperoni pizza,22558380
5,chicken bbq pizza,17622540


## Task 2

Оцените средний срок, который проходит от захода пользователя на сайт до совершения покупки.

In [153]:
first_event_by_user = (
    df_web_logs
    .assign(time = df_web_logs.date.dt.date)
    .groupby(['user_id', 'time'], as_index=False)
    .agg(first_event = ('date', 'min'))
)

purchase_by_user = (
    df_sales
    .assign(time = df_sales.date.dt.date)
    .groupby(['user_id', 'time'], as_index=False)
    .agg(sales_event = ('date', 'min'))
)

(
    first_event_by_user
    .merge(purchase_by_user, how='left', left_on=['user_id', 'time'], right_on=['user_id', 'time'])
    .assign(time_diff = lambda x: (x.sales_event - x.first_event),
            hour_differenece = lambda x: (x.sales_event - x.first_event).astype('timedelta64[h]')
    )
    .query("hour_differenece < 2")
    .time_diff.mean()
)

Timedelta('0 days 00:16:30.281660512')

## Task 3

Какая доля пользователей, совершивших покупку в феврале, совершила покупку и в марте?

In [179]:
users_fp = (
    df_sales
    .groupby('user_id', as_index=False)
    .agg(fp = ('date', 'min'))
    .assign(cohort = lambda x: x.fp.dt.to_period('M'))
)

add_fp_info = (
    df_sales
    .merge(
        users_fp, 
        how = 'left',
        left_on = 'user_id',
        right_on = 'user_id'
    )
    .assign(
        month = lambda x: x.date.dt.to_period('M'),
        month_index = lambda x: 'M' + (x.month.astype('int') - x.cohort.astype('int')).astype('str')
    )
    .groupby(['cohort', 'month_index'], as_index=False)
    .agg(users = ('user_id', 'nunique'))
    .pivot(columns = 'month_index', index = 'cohort', values = 'users')
    .reset_index()
)

In [186]:
(
    add_fp_info
    .assign(
        M0 = add_fp_info.M0 / add_fp_info.M0,
        M1 = round(add_fp_info.M1 / add_fp_info.M0, 3),
        M2 = round(add_fp_info.M2 / add_fp_info.M0, 3)
    )
)

month_index,cohort,M0,M1,M2
0,2022-02,1.0,0.658,0.004
1,2022-03,1.0,0.004,NaN
2,2022-04,1.0,NaN,NaN
